<a href="https://www.kaggle.com/code/isabbaggin/housing-price-prediction?scriptVersionId=147361986" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import pandas as pd

In [2]:
data_path='/kaggle/input/house-rent-prediction-dataset/House_Rent_Dataset.csv'

In [3]:
model_data=pd.read_csv(data_path)

In [4]:
model_data.isnull().sum() #check if there are any missing values---------->here none

Posted On            0
BHK                  0
Rent                 0
Size                 0
Floor                0
Area Type            0
Area Locality        0
City                 0
Furnishing Status    0
Tenant Preferred     0
Bathroom             0
Point of Contact     0
dtype: int64

In [5]:
model_data.columns

Index(['Posted On', 'BHK', 'Rent', 'Size', 'Floor', 'Area Type',
       'Area Locality', 'City', 'Furnishing Status', 'Tenant Preferred',
       'Bathroom', 'Point of Contact'],
      dtype='object')

In [6]:
features=['BHK','Size','Furnishing Status','Bathroom'] #ADD Area Type, Floor and Area Locality if similar

In [7]:
from sklearn.preprocessing import LabelEncoder #using label encoder to map to same values everytime


In [8]:
#model_data['Area Type']=model_data['Area Type'].replace(['Super Area','Carpet Area'],[1,0])
model_data['Furnishing Status']=model_data['Furnishing Status'].replace(['Furnished','Semi-Furnished','Unfurnished'],[2,1,0])
#model_data['Area Type']=pd.to_numeric(model_data['Area Type'],errors='coerce')
model_data['Furnishing Status']=pd.to_numeric(model_data['Furnishing Status'],errors='coerce')

In [9]:
#left_categorical_columns=['Floor','Area Locality']  #convert dtype=object to dtype=category for using category codes to convert object to number
#model_data['Floor']=model_data['Floor'].astype('category')
#all_categories1=model_data['Floor'].unique()
#encoder1=LabelEncoder()
#encoder1.fit(all_categories1)
#model_data['Floor']=encoder1.transform(model_data['Floor'])
    #X[categorical_columns]=X[categorical_columns].apply(lambda x:x.cat.codes)
    
#model_data['Area Locality']=model_data['Area Locality'].astype('category')
#all_categories2=model_data['Area Locality'].unique()
#encoder2=LabelEncoder()
#encoder2.fit(all_categories2)
#model_data['Area Locality']=encoder2.transform(model_data['Area Locality'])
#X['Area Locality'].dtype

In [10]:
model_data=model_data.dropna(axis=0)
model_data.isnull().sum()

Posted On            0
BHK                  0
Rent                 0
Size                 0
Floor                0
Area Type            0
Area Locality        0
City                 0
Furnishing Status    0
Tenant Preferred     0
Bathroom             0
Point of Contact     0
dtype: int64

In [11]:
X=model_data[features]
y=model_data.Rent  

In [12]:
from sklearn.model_selection import train_test_split
train_X, val_X, train_y, val_y=train_test_split(X,y,random_state=1)  #split train and test data
train_X

,BHK,Size,Furnishing Status,Bathroom
1624,1,800,1,1
456,1,500,0,1
3609,2,600,0,2
2707,3,1600,2,2
1012,3,1260,0,3
...,...,...,...,...
2895,1,450,0,1
2763,1,545,2,1
905,3,806,1,2
3980,2,1600,0,3


In [13]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

In [14]:
def get_mae(max_leaf_nodes,train_X,val_X,train_y,val_y):
    m_data=RandomForestRegressor(max_leaf_nodes=max_leaf_nodes,random_state=1)
    m_data.fit(train_X,train_y)
    val_predictions=m_data.predict(val_X)
    mae=mean_absolute_error(val_y,val_predictions)
    return(mae)

In [15]:
import numpy as np
best_leaf=0
best_mae=500000
nodes=np.arange(2,2000,200)
for max_leaf in nodes:
    possible_mae=get_mae(max_leaf,train_X,val_X,train_y,val_y)
    if(possible_mae<best_mae):
        best_leaf=max_leaf
best_no_of_leaves=best_leaf


In [16]:
print(best_no_of_leaves)


1802


In [17]:
train_to_rf=RandomForestRegressor(max_leaf_nodes=best_no_of_leaves,random_state=1)
train_to_rf.fit(X,y)
val_predicted_from_rf=train_to_rf.predict(val_X)
improved_mae=mean_absolute_error(val_predicted_from_rf,val_y)
improved_mae

14049.961045656775

In [18]:
print(val_y)

230      18000
4500     10500
3277     16000
4045     13000
1561      9500
         ...  
1614      7500
829     130000
788     200000
1247    180000
3070      8000
Name: Rent, Length: 1187, dtype: int64


In [19]:
print(val_predicted_from_rf)

[ 23394.66666667  21210.33603733  21777.23834584 ... 119559.24809259
  89212.5515873    8544.69206515]


In [20]:
def predict_rent(bhk,size,furnishing_status,bath): #area type, add floor, area locality
    x=np.zeros(len(X.columns))
    x[0]=bhk
    x[1]=size
    #x[2]=int(floor)
    #x[2]=area_type
    #x[4]=area_locality
    x[2]=furnishing_status
    x[3]=bath
    return(train_to_rf.predict([x]))

In [21]:
data=[['1 out of 3','Carpet Area','Bandel','Furnished']]
df=pd.DataFrame(data,columns=['Floor','Area Type','Area Locality','Furnishing Status'],dtype='category')
#df['Floor']=encoder1.transform(df['Floor'])
#df['Area Locality']=encoder2.transform(df['Area Locality'])
df['Area Type']=df['Area Type'].replace(['Super Area','Carpet Area'],[1,0])
df['Furnishing Status']=df['Furnishing Status'].replace(['Furnished','Semi-Furnished','Unfurnished'],[2,1,0])
predict_rent(3,1000,df['Furnishing Status'],2)

/tmp/ipykernel_20/4197940834.py:8: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  x[2]=furnishing_status
/opt/conda/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


array([32640.40584416])